In [ ]:
import pandas as pd
app_energy = pd.read_csv("/Users/achirukuri/Downloads/energydata_complete.csv")
app_energy.head()

In [ ]:
app_energy.shape

In [ ]:
y = app_energy.head(20)
y.head()

In [ ]:
y.plot() # can enhance grid and see the changes

## Convert the date into date time and save them in a separate columns

In [ ]:
app_energy.head()

In [ ]:
type(app_energy['date'])
import dask.dataframe as dd
app_energy['date'] = pd.to_datetime(app_energy['date'])
app_energy.set_index(app_energy['date'],inplace=True)
app_energy.index
data_sample_from_pd = dd.from_pandas(app_energy, npartitions=1)
data_sample_from_pd.index.head()

In [ ]:
weekly = app_energy.resample('W').sum()
weekly.plot(style = [':', '--', '-'])

In [ ]:
daily = app_energy.resample('D').sum()
daily.rolling(30, center = True).sum().plot(style =[':', '--', '-'])

## Part 2 Code

In [ ]:
import numpy as np 
import pandas as pd 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, model_selection, metrics
import warnings
warnings.filterwarnings("ignore")

In [ ]:
app_energy.info()

## Splitting the data into training and splitting

In [ ]:
from sklearn.model_selection import train_test_split
data = app_energy
# 75% of the data is usedfor the training of the models and the rest is used for testing
train, test = train_test_split(data,test_size=0.25,random_state=40)

In [ ]:
train.shape

## Dividing the columns based on their features

In [ ]:
# Dividing the columns based on their features like temperature, humidity anf weather (outside the house) and lights, random variables and appliances.

temp = ["T1","T2","T3","T4","T5","T6","T7","T8","T9"]

hum = ["RH_1","RH_2","RH_3","RH_4","RH_5","RH_6","RH_7","RH_8","RH_9"]

weather = ["T_out", "Tdewpoint","RH_out","Press_mm_hg",
                "Windspeed","Visibility"] 
light = ["lights"]

randoms = ["rv1", "rv2"]

target = ["Appliances"]


## Separating the target variables

In [ ]:
feature_vars = train[temp + hum + weather + light + randoms ]
target_vars = train[target]

In [ ]:
feature_vars.drop(columns = ['lights'], axis = 1, inplace = True)

In [ ]:
feature_vars.head(2)

## Time series representation of the entire dataset

In [ ]:
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

# To understand the timeseries variation of the applaince energy consumption
Visulaization = go.Scatter( x= data.date  ,  mode = "lines", y = data.Appliances )
viz_layout = go.Layout(title = 'Appliance energy consumption of entire dataset' , xaxis=dict(title='Date'), yaxis=dict(title='(Wh)'))
fig = go.Figure(data=[Visulaization],layout=viz_layout)

iplot(fig)

Creating Weekday Variable for timeseries representation

In [ ]:
data['WEEKDAY'] = ((pd.to_datetime(data['date']).dt.dayofweek)//5).astype(float)
# There are 5472 weekend recordings 
data['WEEKDAY'].value_counts()

In [ ]:
data.head()

## Time series representation of all weekdays

In [ ]:
weekday_timeseries =  data[data['WEEKDAY'] == 0]
# To understand the timeseries variation of the applaince energy consumption
visData = go.Scatter( x= weekday_timeseries.date  ,  mode = "lines", y = weekday_timeseries.Appliances )
layout = go.Layout(title = 'Appliance energy consumption -  weekdays' , xaxis=dict(title='Date'), yaxis=dict(title='(Wh)'))
fig = go.Figure(data=[visData],layout=layout)

iplot(fig)

## Time series representation of all Weekends

In [ ]:
weekend_timeseries =  data[data['WEEKDAY'] == 1]
# To understand the timeseries variation of the applaince energy consumption
visData = go.Scatter( x= weekend_timeseries.date  ,  mode = "lines", y = weekend_timeseries.Appliances )
layout = go.Layout(title = 'Appliance energy consumption - weekends ' , xaxis=dict(title='Date'), yaxis=dict(title='(Wh)'))
fig = go.Figure(data=[visData],layout=layout)

iplot(fig)

## Histogram density function

In [ ]:
# Histogram of all the features to understand the distribution
feature_vars.hist(bins = 20 , figsize= (12,16)) ;


#### RH_6, RH_out, visibility, windspeed and T9 have irregular distributions 

Pairwise plots distribution

In [ ]:
sns.pairplot(feature_vars);

In [ ]:
# focussed displots for RH_6 , RH_out , Visibility , Windspeed, T9 due to irregular distribution
f, ax = plt.subplots(2,2,figsize=(12,10))
den1 = sns.distplot(feature_vars["RH_6"],bins=10, ax= ax[0][0])
den2 = sns.distplot(feature_vars["RH_out"],bins=10, ax=ax[0][1])
den3 = sns.distplot(feature_vars["Visibility"],bins=10, ax=ax[1][0])
den4 = sns.distplot(feature_vars["Windspeed"],bins=10, ax=ax[1][1])

In [ ]:
# Distribution of values in Applainces column
Appliances_dis = plt.figure(figsize=(12,5))
plt.xlabel('Appliance consumption in Whatts')
plt.ylabel('Frequency')
sns.distplot(target_vars , bins=10 ) ;
# postive skewed 

In [ ]:
# count of appliances grater than 200 wh
data['Appliances'][data['Appliances'] > 200].count()

In [ ]:
# feature variables have all columns and target_vars have the appliance info. 
print('Percentage of the appliance consumption is less than 200 Wh')
print(((target_vars[target_vars <= 200].count()) / (len(target_vars)))*100 )

### More than 90% of the appliances consume less than 200 wh.

## Correlation Matrix

In [ ]:
corr_plot = train[temp + hum + weather +target+randoms]
corr = corr_plot.corr()
# Mask the repeated values
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
  
f, ax = plt.subplots(figsize=(16, 14))
#Generate Heat Map, allow annotations and place floats in map
sns.heatmap(corr, annot=True, fmt=".2f" , mask=mask,)
    #Apply xticks
plt.xticks(range(len(corr.columns)), corr.columns);
    #Apply yticks
plt.yticks(range(len(corr.columns)), corr.columns)
    #show plot
plt.show()


#### RH6 and T9 are highly correlated so remove the highly correlated variables

In [ ]:
corrmatplot = data.corr() 
  
f, ax = plt.subplots(figsize =(9, 8)) 
sns.heatmap(corrmatplot, ax = ax, cmap ="YlGnBu", linewidths = 0.1) 

## Identifying the variables which are highly correlated

In [ ]:
data.corr().unstack().sort_values(ascending = False).drop_duplicates()

In [ ]:
app_energy.head()

## Average of temperatures and humidity 

In [ ]:
app_energy.head()

In [ ]:
'''df = energy_dat
df['avg_temp'] = df[['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out']].mean(axis = 1)
df['avg_RH'] = df[['RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out']].mean(axis = 1)
df.head()'''

In [ ]:
feature_vars.columns


In [ ]:
train.columns


## Splitting the data into training and test dataset

In [ ]:
x_train = train[feature_vars.columns]
y_train = train[target_vars.columns]

In [ ]:
x_test = test[feature_vars.columns]
y_test = test[target_vars.columns]

#### Four columns have a high degree of correlation with T9 - T3,T5,T7,T8 also T6 & T_Out.
#### rh6 most negatively correlated and T9 most positively correlated.

In [ ]:
x_train.drop(["rv1","rv2","Visibility","RH_6","T9"],axis=1 , inplace=True)

In [ ]:
x_test.drop(["rv1","rv2","Visibility","RH_6", "T9"], axis=1, inplace=True) 

In [ ]:
x_train.columns

In [ ]:
x_test.columns

## Normalizing the dataset and preprocessing using Scaling method

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

sc=StandardScaler()
n = Normalizer()

# Create test and training set by including Appliances column

train = train[list(x_train.columns.values) + target ]

test = test[list(x_test.columns.values) + target ]

# Create dummy test and training set to hold scaled values

sc_train = pd.DataFrame(columns=train.columns , index=train.index)

sc_train[sc_train.columns] = n.fit_transform(train)

sc_test= pd.DataFrame(columns=test.columns , index=test.index)

sc_test[sc_test.columns] = n.fit_transform(test)


In [ ]:
x_train =  sc_train.drop(['Appliances'] , axis=1)
y_train = sc_train['Appliances']

x_test =  sc_test.drop(['Appliances'] , axis=1)
y_test = sc_test['Appliances']

In [ ]:
x_test.to_csv('/Users/achirukuri/Desktop/test_x.csv')

In [ ]:
y_test.to_csv('/Users/achirukuri/Desktop/test_y.csv')

In [ ]:
x_train.to_csv('/Users/achirukuri/Desktop/train_x.csv')

In [ ]:
y_test.to_csv('/Users/achirukuri/Desktop/test_y.csv')

In [ ]:
y_train

## Using SKlearn to implement all the models

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
from sklearn import neighbors
from sklearn.svm import SVR

In [ ]:
from sklearn.ensemble import RandomForestRegressor
randomforest = RandomForestRegressor(random_state = 42)
from pprint import pprint

In [ ]:
print('Parameters currently in use:\n')
pprint(randomforest.get_params())

## Using Random Search CV method for tuning the methods

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
randomforest = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_model = RandomizedSearchCV(estimator = randomforest, param_distributions = grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_model.fit(x_train, y_train)

In [ ]:
rf_model.best_params_

In [ ]:
best_grid = rf_model.best_params_
#grid_accuracy = evaluate(best_grid, test_X, test_y)

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
randomforest = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
randomforest.fit(x_train, y_train);

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(x_test)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

## For retrieving the random forest tree graph

In [ ]:
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 7)
rf_small.fit(x_train, y_train)
# Extract the small tree
tree_small = rf_small.estimators_[9] 


In [ ]:
# Limit depth of tree to 3 levels
rf_small = ExtraTreesRegressor(n_estimators=10, max_depth = 4)
rf_small.fit(train_X, train_y)
# Extract the small tree
tree_small = rf_small.estimators_[9] 
# Save the tree as a png image


In [ ]:
model = rf.fit(train_X, train_y)
results = rf.predict(test_)

In [ ]:
MSE  = mean_squared_error(test_X, results)  #Calculate Mean Squared Error
RMSE = sqrt(MSE)
print ("RMSE : ",RMSE)

In [ ]:
models = [
           ['Lasso: ', Lasso()],
           ['Ridge: ', Ridge()],
           ['KNeighborsRegressor: ',  neighbors.KNeighborsRegressor()],
           ['SVR:' , SVR(kernel='rbf')],
           ['RandomForest ',RandomForestRegressor()],
           ['ExtraTreeRegressor :',ExtraTreesRegressor()],
           ['GradientBoostingRegressor: ', GradientBoostingRegressor()] ,
           ['XGBRegressor: ', xgb.XGBRegressor()] ,
           ['MLPRegressor: ', MLPRegressor(  activation='relu', solver='adam',learning_rate='adaptive',max_iter=1000,learning_rate_init=0.01,alpha=0.01)]
         ]

In [ ]:
import time
from math import sqrt
from sklearn.metrics import mean_squared_error

model_data = []
for name,curr_model in models :
    curr_model_data = {}
    curr_model.random_state = 78
    curr_model_data["Name"] = name
    start = time.time()
    curr_model.fit(train_X,train_y)
    end = time.time()
    curr_model_data["Train_Time"] = end - start
    curr_model_data["Train_R2_Score"] = metrics.r2_score(train_y,curr_model.predict(train_X))
    curr_model_data["Test_R2_Score"] = metrics.r2_score(test_y,curr_model.predict(test_X))
    curr_model_data["Test_RMSE_Score"] = sqrt(mean_squared_error(test_y,curr_model.predict(test_X)))
    curr_model_data["Train_RMSE_Score"] = sqrt(mean_squared_error(train_y,curr_model.predict(train_X)))
    model_data.append(curr_model_data)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
model_data

In [ ]:
df = pd.DataFrame(model_data)

In [ ]:
df.iloc[1:,]

## Created a dataframe and the data points are the values which are obtained from the models in the R code except extrta regressor

In [ ]:
import pandas as pd

In [ ]:
data = [['MARS', 0.075, 0.995], ['Ridge', 0.0334, 0.909], ['Lasso', 0.0334, 0.909], ['GBM', 0.004, 0.998], ['SVM', 0.0096,0.993], 
       ['KNN',0.02470946, 0.95065926], ['Decision Trees', 0.03289199, 0.90866495], ['PLS', 0.03372,0.9079], ['Enet', 0.0354,0.9038],
       ['Extratree Regressor', 0.004421, 0.998350], ['Random Forest', 0.0063, 0.9968]]
df = pd.DataFrame(data, columns = ['Name', 'RMSE', 'R2']) 


In [ ]:
df = df.sort_values(by = ['R2'])

In [ ]:
df.plot(x="Name", y=['R2' , 'RMSE'], kind="bar" ,
        title = 'RMSE, R2 Score Results' , figsize= (6,6)) ;

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [{
              'max_depth': [80, 150, 200,250],
              'n_estimators' : [100,150,200,250],
              'max_features': ["auto", "sqrt", "log2"]
            }]
reg = ExtraTreesRegressor(random_state=40)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = reg, param_grid = param_grid, cv = 5, n_jobs = -1 , scoring='r2' , verbose=2)
grid_search.fit(train_X, train_y)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_search.best_estimator_.score(train_X,train_y)

In [ ]:
grid_search.best_estimator_.score(test_X,test_y)


In [ ]:
np.sqrt(mean_squared_error(test_y, grid_search.best_estimator_.predict(test_X)))

In [ ]:
tuning xtree regressor : 
    R2: 0.998901
    RMSE: 0.003607

In [ ]:
np.sqrt(mean_squared_error(test_y,grid_search.best_estimator_.predict(test_X)))

## Feature Importance

In [ ]:
feature_indices = np.argsort(grid_search.best_estimator_.feature_importances_)

In [ ]:
importances = grid_search.best_estimator_.feature_importances_
indices = np.argsort(importances)[::-1]
names = [train_X.columns[i] for i in indices]
# Create plot
plt.figure(figsize=(10,6))

# Create plot title
plt.title("Feature Importance")

# Add bars
plt.bar(range(train_X.shape[1]), importances[indices])

# Add feature names as x-axis labels
plt.xticks(range(train_X.shape[1]), names, rotation=90)

# Show plot
plt.show()

In [ ]:
train_important_feature = train_X[names[0:5]]
test_important_feature = test_X[names[0:5]]

In [ ]:
from sklearn.base import clone
cloned_model = clone(grid_search.best_estimator_)
cloned_model.fit(train_important_feature , train_y)